# Scissor Jack Optimizer

In [ ]:
# import packages here
using SNOW
using Plots

In [ ]:
# Definition of structs for components of jack
"""
Diagonal member
This is the struct object for the diagonal members. It assumes that they are all U-cross_bar_bearing_stress

t is the thickness of the diagonal member in inches
w is the width of the diagonal member in inches
S_y is the yield strength of the diagonal member in psi
ppi is the price per inch of the diagonal member in USD/inch
"""
struct DiagonalMember
    t::Float32
    w::Float32
    S_y::Float32
    ppi::Float32
end

"""
Crossbar
This is the struct object for the crossbars. It assumes that they are all square bars.

t is the thickness of the crossbar in inches
w is the width of the crossbar in inches
S_y is the yield strength of the crossbar in psi
E is the modulus of elasticity of the crossbar in psi
ppi is the price per inch of the crossbar in USD/inch
"""
struct Crossbar
    t::Float32
    w::Float32
    S_y::Float32
    E::Float32
    ppi::Float32
end

"""
Pin
This is the struct object for the pins.

d is the diameter of the pin in inches
S_y is the yield strength of the pin in psi
p is the price for a singular pin in USD
"""
struct Pin
    t::Float32
    w::Float32
    S_y::Float32
    E::Float32
    ppi::Float32
end

# definition of Best Solution So Far (BSSF) struct
"""
BSSF
This is the struct object for the Best Solution So Far (BSSF)

p is the price
diag is the struct object for the diagonal member
cross is the struct object for the crossbar
pin is the struct object for the pin
len_d is the length of the diagonal members
len_cb is the length of the crossbar
hole_d is the length from the center of the hole to the edge of the member for the diagonal members
hole_cb is the length from the center of the hole to the edge of the member for the crossbar
"""
mutable struct BSSF
    p::Float16
    diag::DiagonalMember
    cross::Crossbar
    pin::Pin
    len_d::Float32
    len_cb::Float32
    hole_d::Float32
    hole_cb::Float32
end

BSSF

## Pseudocode for search method

for pin in pins

&nbsp; &nbsp; &nbsp; &nbsp; for diag in diagonal_members

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; for cross in crossbars

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; objective = make objective(cross, diag, pin, applied_force, end_condition_constant)

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; other optimizer inputs

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; xopt, fopt, info = minimize(objective!, x0, ng, lx, ux, lg, ug, options)

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; get price from outputs

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; if price is < bssf.price

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; update bssf

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; end

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; end

&nbsp; &nbsp; &nbsp; &nbsp; end

end

display bssf
